In [ ]:
# sudo apt-get update
# sudo apt install chromium-chromedriver
# sudo apt install chromium-bsu

In [ ]:
# from dataclasses import dataclass
# import pandas as pd
# from sqlalchemy.util import dataclass_fields

# from urllib.parse import urljoin
# import requests
# from bs4 import BeautifulSoup
# import re
# from IPython.display import display, HTML


In [ ]:
import logging

import sqlite3
from fastcore.basics import patch_to

# download_url
import time
import selenium




In [ ]:
class Crawler:

    sqlite_driver_db: str
    sqlit_con: sqlite3.connect

    url_to_visit_ls: list[str]

    url_visited_ls: list[str] = []
    article_ls : list[str] = []

    def __init__(self,
                 sqlite_driver_db: str,
                 url_to_visit_ls: list[str] = [],
                 base_url: str = None):

        self.sqlite_driver_db = sqlite_driver_db
        self.sqlit_con = sqlite3.connect(self.sqlite_driver_db)
        self.base_url = base_url
        self.url_to_visit_ls = url_to_visit_ls


    
    def _add_url_to_visit(self, url ):
      if url not in self.url_visited_ls and url not in self.url_to_visit_ls:
        self.url_to_visit_ls.append(url)


In [ ]:
logging.basicConfig(
    format='%(asctime)s %(levelname)s:%(message)s',
    level=logging.INFO)

cd = Crawler(sqlite_driver_db = 'kb_scrape', base_url = '')

In [ ]:
@patch_to(Crawler)
def run(self: Crawler, debug_prn: bool = False):
    
    counter = 0
    upload_counter = 0
    num_upload = 20
    offset = 0
    max_upload = 10000

    while self.url_to_visit_ls and counter <= max_upload:
        url = self.url_to_visit_ls.pop(0)
        
        logging.info(f'Crawling: {url}')
        
        try:
            # self.crawl(url, debug_prn)

            pass
            
        except Exception:
            logging.exception(f'Failed to crawl: {url}')

        finally:
            self.url_visited_ls.append(url)

        counter += 1

    #     if counter == 10:
    #     self.update_exist_ls()

    #     if counter % num_upload == 0:
    #     offset = num_upload * upload_counter
    #     df = pd.DataFrame(self.article_ls).iloc[offset:]

    #     print(f"rows uploading {offset} - {counter}")

    #     if upload_counter == 0:
    #         self.insert_into_sql(df, is_reset=True)
    #     else:
    #         self.insert_into_sql(df, is_reset=False)

    #     upload_counter += 1

    # df = pd.DataFrame(self.article_ls).iloc[offset:]

    # self.insert_into_sql(df, is_reset=False)

    print('RUN: done crawling')

    return self

In [ ]:
logging.basicConfig(
    format='%(asctime)s %(levelname)s:%(message)s',
    level=logging.INFO)

url_to_visit_ls = [
    'https://domo-support.domo.com/s/article/360047400753?language=en_US', 'https://domo-support.domo.com/s/topic/0TO5w000000ZaoEGAS/sharing-access-to-cards-and-pages?language=en_US']

cd = Crawler(sqlite_driver_db='kb_scrape', base_url='', url_to_visit_ls= url_to_visit_ls)

cd.run()

2023-01-25 02:42:51,076 INFO:Crawling: https://domo-support.domo.com/s/article/360047400753?language=en_US
2023-01-25 02:42:51,077 INFO:Crawling: https://domo-support.domo.com/s/topic/0TO5w000000ZaoEGAS/sharing-access-to-cards-and-pages?language=en_US


RUN: done crawling


<__main__.Crawler>

In [ ]:
@patch_to(Crawler)
def download_url(self: Crawler, url, debug_prn: bool = False):

  options = webdriver.ChromeOptions()
  options.add_argument("-headless")
  options.add_argument("-no-sandbox")
  options.add_argument("-disable-dev-shm-usage")

  wd = webdriver.Chrome("chromedriver", options=options)

  if debug_prn:
        print(f'downloading data from {url}')

  wd.get(url)
  time.sleep(5)  # sleeps for 5 seconds to give page time to load
  source = wd.page_source

  return source


In [ ]:
import time
import selenium
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument("-headless")
options.add_argument("-no-sandbox")
options.add_argument("-disable-dev-shm-usage")

wd = webdriver.Chrome("chromedriver", options=options)

wd.get(url)


WebDriverException: Message: Service chromedriver unexpectedly exited. Status code was: 1


In [ ]:
@patch_to(Crawler)
def crawl(self : Crawler, url, debug_prn: bool = False):
    
    if debug_prn:
        print(f"CRAWL:  starting crawl - {url}")

    html = self.download_url(url, debug_prn=debug_prn)

    # article = self.get_article(html, url)

    linked_urls_ls = None
    # linked_urls_ls = article.get('linked_urls_ls')

    if not linked_urls_ls or len(linked_urls_ls) == 0:
        return

    for url in linked_urls_ls:
        self._add_url_to_visit(url)


In [ ]:
# def update_exist_ls(self):
#       """updates crawler's visited_urls based on my database of existing kbs"""
#       cursor = self.sqlit_con.execute('''SELECT url FROM kb''')

#       for row in cursor:
#         self.visited_urls.append(row[0])



#     def get_linked_urls(self, html, debug_prn: bool = False):
#       import urllib.parse as url_parse
#       from bs4 import BeautifulSoup

#       soup = BeautifulSoup(html, 'html.parser')

#       linked_urls = []
#       for link in soup.find_all('a'):
#           path = link.get('href')

#           if not path:
#             continue

#           if path.startswith('/'):
#               path = url_parse.urljoin(self.base_url, path)

#           if path.startswith(self.base_url):
#             linked_urls.append(path)

#       return linked_urls



#     def insert_into_sql(self, article_df, is_reset: bool = False):
#       con = sqlite3.connect(
#           "/content/drive/MyDrive/Colab Notebooks/webscrape_domo/my_db2.db")

#       print(article_df.__dict__)

#       article_df.linked_urls = ", ".join(article_df.linked_urls)

#       # display(HTML(article_df.to_html()))

#       if is_reset:
#         article_df.to_sql("kb", con, if_exists="replace")

#       article_df.to_sql("kb", con, if_exists="append")

#     def get_article(self, html, url):
#       soup = BeautifulSoup(html, 'html.parser')

#       form_labels = soup.find_all(class_=['slds-form-element'])

#       linked_urls_ls = self.get_linked_urls(html)

#       article = {"url": url,
#                  "linked_urls": linked_urls_ls}

#       for form in form_labels:
#         rows = list(form.strings)

#         if len(rows) >= 2:
#           title = rows[0]
#           value_ls = rows[1:]

#           value_tx = " ".join(value_ls)

#           value_clean = re.sub(r'( \n.?)+', r'\r', value_tx)
#           article.update({title: value_clean})

#       if article.get('Article Body'):
#         self.article_ls.append(article)
#       else:
#         self.add_url_to_visit(url)

#       return article





In [ ]:
import platform
platform.platform()

'Linux-5.4.0-1100-azure-x86_64-with-glibc2.31'